# Multiple Models by Versions

## Model predictions

In [1]:
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.serializers import CSVSerializer

In [2]:
# load test data
test = pd.read_csv("test.csv")
test = test.iloc[:,1:].values

In [3]:
# Load model
xgb_model = sagemaker.predictor.Predictor(endpoint_name="xgboost-loan-byVersion")
xgb_model.serializer = CSVSerializer()

In [4]:
# target_variant=None, endpoint distributes the test data based on the model weight.
pred1 = xgb_model.predict(test, target_variant=None).decode('utf-8')
print(pred1[:10])

1.0,1.0,1.


In [5]:
pred1 = np.fromstring(pred1, sep=',')
pred1[:5]

array([1., 1., 1., 1., 1.])

In [6]:
# target_variant=0.90-1
pred2 = xgb_model.predict(test, target_variant="version-0-90-1").decode('utf-8')  
pred2 = np.fromstring(pred2, sep=',')

# target_variant=1.3-1
pred3 = xgb_model.predict(test, target_variant="version-1-3-1").decode('utf-8')
pred3 = np.fromstring(pred3, sep=',')

In [7]:
print(f"""
The number of predictions when:
None = 0.90-1 = 1.3-1: {(sum((pred1 == pred2) & (pred2 == pred3)))}
None = 0.90-1 >< 1.3-1: {(sum((pred1 == pred2) & (pred2 != pred3)))}
None >< 0.90-1 = 1.3-1: {(sum((pred1 != pred2) & (pred2 == pred3)))}
None = 1.3-1 >< 0.90-1: {(sum((pred1 == pred3) & (pred2 != pred3)))}
""")


The number of predictions when:
None = 0.90-1 = 1.3-1: 95
None = 0.90-1 >< 1.3-1: 0
None >< 0.90-1 = 1.3-1: 0
None = 1.3-1 >< 0.90-1: 0



In [ ]:
# Delete the endpoints if not needed anymore.
# xgb_model.delete_endpoint()